In [1]:
import os
# import threading
# from pyspark import SparkContext, SparkConf

csv_store_path = os.environ['GP_HIST_PATH']
#csv_store_path = os.environ['my_home_path'] + '\data\space-track-gp-hist-sample'
file_list = 'all_files.txt'
csv_store_path

'D:\\Datasets\\Space-Track gp_history'

In [2]:
# write a file containing all the file names
files = sorted([x for x in os.listdir(f'{csv_store_path}/') if x.endswith(".csv.gz")])
with open(file_list, 'w+') as f:
    for file in files:
        f.write(csv_store_path + '\\' + file + '\n')

In [3]:
# import pandas as pd
# file_path = r'D:\Datasets\Space-Track gp_history\002000553-002100571.csv.gz'
# #file_path = r'D:\Dropbox (University of Michigan)\Milestone 2\data\000000001-000100186.csv.gz'
# df = pd.read_csv(file_path, compression='gzip', low_memory=False)
# df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25)]
# #print(f'File: {file_path}')
# df.NORAD_CAT_ID.to_list()

In [4]:
%%file norad_counts.py
from mrjob.job import MRJob, JSONProtocol
import pandas as pd

class MRNoradCounts(MRJob):
    
    def mapper(self, _, file_path):
        try:
            df = pd.read_csv(file_path, compression='gzip', low_memory=False)
            df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25)]
        except:
            raise Exception(f'Failed to open {file_path}') 
        #print(f'File: {file_path}')
        for norad in df.NORAD_CAT_ID.to_list():
            yield norad, 1
            
    def combiner(self, norad, counts):
        yield norad, sum(counts)
        
    def reducer(self, norad, counts):
        yield norad, sum(counts)
        
if __name__ == "__main__":
    #mr_job = MRNoradCounts()
    #runner = mr_job.make_runner()
    #runner.run()
    MRNoradCounts.run()
#     mr_job = MRNoradCounts(args=[r'all_files.txt',
#                                 #'-r', 'hadoop',
#                                 #'--hadoop-streaming-jar', r'C:\hadoop-3.3.0\share\hadoop\tools\lib\hadoop-streaming-3.3.0.jar',
#                                 #r'>output.txt',
#                                 ])
#     runner = mr_job.make_runner()
#     runner.run()
#     with mr_job.make_runner() as runner:
#         runner.run()
#         with open(r'output.csv', 'w+') as f: 
#             for line in runner.stream_output():
#                 k,v = mr_job.parse_output_line(line)
#                 f.write(f'{k},{v}\n')

Overwriting norad_counts.py


In [5]:
#!python norad_counts.py all_files.txt >output.txt

In [6]:
#python norad_counts.py --hadoop-streaming-jar C:\hadoop-3.3.0\share\hadoop\tools\lib\hadoop-streaming-3.3.0.jar all_files.txt >output.txt